In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path
scripts_path = Path("../Modeling/").resolve()
sys.path.append(str(scripts_path))

In [ ]:
import pickle
from scripts.data_visualiser import *
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.get_statistics import *
from Pipeline.pre_processing_pipeline import *
import numpy as np
import preprocessing_config as config

In [ ]:
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 
from keras.applications.vgg16 import VGG16 
from keras.models import Model
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

## Week 3

In [ ]:
images = load_sentinel_images_temporal('/Users/bhumikasadbhave007/Documents/THWS/Semester-4/MASTER-THESIS/Temporal-Data/test/first-10')

In [ ]:
len(images)

In [ ]:
masked_images = mask_images_temporal(images)

In [ ]:
fields = extract_fields_temporal(images, config.field_size)

In [ ]:
len(fields)

In [ ]:
fields[0][0].shape

In [ ]:
refined_fields = refine_temporal_stack_interval5(fields, config.temporal_stack_size, config.temporal_points)

In [ ]:
len(refined_fields)

In [ ]:
refined_fields[0][0].shape

##### Image Diseased

In [ ]:
np.unique(refined_fields[6][1][:,:,-2])

In [ ]:
visualize_temporal_stack_rgb(refined_fields[8])

In [ ]:
visualize_temporal_stack_ndvi(refined_fields[1])

In [ ]:
visualize_temporal_stack_ci(refined_fields[1])

In [ ]:
visualize_temporal_stack_msi(refined_fields[1])

In [ ]:
visualize_temporal_stack_evi(refined_fields[1])

## Week 4/5

##### Visualisation Bounding Box

In [ ]:
rgb_image = images[2][7][:,:,:3] #[:, :, [2, 1, 0]]  # Red, Green, Blue bands
# rgb_image = sm_rotated[:,:,:3]

# Normalize RGB image for visualization
rgb_normalized = (rgb_image - np.min(rgb_image)) / (np.max(rgb_image) - np.min(rgb_image))
brightness_factor = 5  # >1 to increase brightness, <1 to decrease
rgb_adjusted = np.clip(rgb_normalized * brightness_factor, 0, 1)

# Label connected regions in the mask
labeled_mask = label(images[2][7][:,:,-1])
regions = regionprops(labeled_mask)

# Plot the RGB image
plt.figure(figsize=(10, 10))
plt.imshow(rgb_adjusted)
plt.title("Sentinel RGB Image with Bounding Boxes")

# Draw bounding boxes for each region
for region in regions:
    min_row, min_col, max_row, max_col = region.bbox
    # Draw a rectangle around the region
    plt.plot([min_col, max_col, max_col, min_col, min_col],  # x-coordinates
            [min_row, min_row, max_row, max_row, min_row],  # y-coordinates
            color='red', linewidth=1)

plt.axis("off")
plt.show()

#### Other Stuff

In [ ]:
success = save_field_images_temporal(config.base_directory_temporal, refined_fields)
success

In [ ]:
temporal_images = load_field_images_temporal(config.base_directory_temporal_test)

In [ ]:
len(temporal_images)

In [ ]:
np.unique(temporal_images[14][0][:,:,-2])

In [ ]:
ndvi_images = temporal_differences_with_time(temporal_images,'ndvi')
len(ndvi_images)

In [ ]:
visualize_temporal_differences(ndvi_images[14])

In [ ]:
visualize_temporal_stack_ndvi(temporal_images[14])

In [ ]:
visualize_temporal_stack_msi(temporal_images[4])

In [ ]:
visualize_temporal_stack_rgb(temporal_images[12])

In [ ]:
visualise_all_bands(temporal_images[4][-1])

No Disease: 5 

In [ ]:
visualize_temporal_differences(ndvi_images[5])

In [ ]:
visualize_temporal_stack_ndvi(temporal_images[5])

In [ ]:
np.unique(temporal_images[7][0][:,:,-2])

## Non-temporal images for BASELINE

In [ ]:
simple_images = get_non_temporal_images(temporal_images)

In [ ]:
len(simple_images)

In [ ]:
simple_images[0].shape

In [ ]:
refined_images, field_numbers = refine_chanel_non_temporal(simple_images)

In [ ]:
refined_images[0].shape

In [ ]:
resized_images = []
for img in refined_images:
    resized_images.append(resize_with_padding(img))

In [ ]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(image, model):

    reshaped_img = image.reshape(1,224,224,3) 
    imgx = preprocess_input(reshaped_img)
    features = model.predict(imgx, use_multiprocessing=True)

    return features

In [ ]:
data = {}
i=0
for image in resized_images:
    feat = extract_features(image,model)
    name = field_numbers[i]
    data[name] = feat
    i+=1

In [ ]:
filenames = np.array(list(data.keys()))
feat = np.array(list(data.values()))
feat = feat.reshape(-1,4096)

# # reduce the amount of dimensions in the feature vector
# pca = PCA(n_components=2, random_state=22)
# pca.fit(feat)
# x = pca.transform(feat)

kmeans = KMeans(n_clusters=2, random_state=22)
kmeans.fit(feat)


In [ ]:
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
acc = get_accuracy(filenames, kmeans.labels_, config.labels_path)
acc

In [ ]:
groups[1]